In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

### Importing the grid search results from the simulator

In [2]:
file_path = "gem5-aladdin_data/stencil_stencil3d_results.csv"
results_df = pd.read_csv(file_path, delimiter=",").drop_duplicates()

In [3]:
results_df.columns

Index(['tlb_page_size', 'tlb_entries', 'tlb_assoc', 'cycle_time',
       'tlb_max_outstanding_walks', 'tlb_miss_latency', 'cache_line_sz',
       'cache_assoc', 'cache_queue_size', 'tlb_hit_latency', 'pipelined_dma',
       'cache_size', 'pipelining', 'cache_hit_latency', 'cache_bandwidth',
       'tlb_bandwidth', 'success', 'cycle', 'power', 'area'],
      dtype='object')

In [4]:
X_data_np = results_df[['tlb_page_size', 'tlb_entries', 'tlb_assoc', 'cycle_time',
       'tlb_max_outstanding_walks', 'tlb_miss_latency', 'cache_line_sz',
       'cache_assoc', 'cache_queue_size', 'tlb_hit_latency', 'pipelined_dma',
       'cache_size', 'pipelining', 'cache_hit_latency', 'cache_bandwidth',
       'tlb_bandwidth']].values.astype(np.float32)

y_data_np = results_df['power'].values.astype(np.float32)

In [5]:
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X_data_np, 
                                                                y_data_np, test_size=0.33, random_state=42)

### Converting numpy arrays to torch tensors

In [6]:
X_train = torch.tensor(X_train_np)
X_test = torch.tensor(X_test_np)
y_train = torch.tensor(y_train_np)
y_test = torch.tensor(y_test_np)

# Neural Network

A simple feed forward neural network

In [7]:
class FFNet(nn.Module):

    def __init__(self):
        super(FFNet, self).__init__()
        
        self.input_ch = 16
        self.output_ch = 1
        
        ######## Hidden layers ########
        
        #   linear functions
        self.lf1_output_ch = 100
        self.lf1 = nn.Linear(self.input_ch, self.lf1_output_ch)
        
#         self.lf2_output_ch = 100
#         self.lf2 = nn.Linear(self.lf1_output_ch, self.lf2_output_ch)
        
        self.lf3_output_ch = 50
        self.lf3 = nn.Linear(self.lf1_output_ch, self.lf3_output_ch)
        
#         self.lf4_output_ch = 50
#         self.lf4 = nn.Linear(self.lf3_output_ch, self.lf4_output_ch)
        
#         self.lf5_output_ch = 50
#         self.lf5 = nn.Linear(self.lf4_output_ch, self.lf5_output_ch)
        
#         self.lf6_output_ch = 50
#         self.lf6 = nn.Linear(self.lf5_output_ch, self.lf6_output_ch)
        
#         self.lf7_output_ch = 25
#         self.lf7 = nn.Linear(self.lf6_output_ch, self.lf7_output_ch)
        
#         self.lf8_output_ch = 25
#         self.lf8 = nn.Linear(self.lf7_output_ch, self.lf8_output_ch)
        
#         self.lf9_output_ch = 25
#         self.lf9 = nn.Linear(self.lf8_output_ch, self.lf9_output_ch)
        
        self.lf10_output_ch = 10
        self.lf10 = nn.Linear(self.lf3_output_ch, self.lf10_output_ch)
        
        #   linear function
        self.lff = nn.Linear(self.lf10_output_ch, self.output_ch)
        
        ######## Functions ########
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, X):
        
        out = self.lf1(X)
        out = self.relu(out)
        out = self.lf3(out)
        out = self.relu(out)
        out = self.lf10(out)
        out = self.relu(out)
        out = self.lff(out)
        out = self.relu(out)
        
        return out

# Training

In [8]:
ffnet_model = FFNet()
optimizer = torch.optim.Adam(ffnet_model.parameters(), lr=0.00001)
criterion = nn.MSELoss()

In [9]:
# Sets the module in training mode.
ffnet_model.train()

num_epochs = 100000

for epoch in range(num_epochs):
    
    optimizer.zero_grad()
    
    # Forward pass
    y_pred = ffnet_model(X_train)
    
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
    
    if ((epoch+1) % ((num_epochs) * 0.1) == 0) or (epoch == 0):
        print('Epoch {} {}%: train loss: {}'.format(epoch+1, (epoch+1)/num_epochs*100,loss.item()))
    
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 1 0.001%: train loss: 233448.421875
Epoch 10000 10.0%: train loss: 86.77014923095703
Epoch 20000 20.0%: train loss: 84.00674438476562
Epoch 30000 30.0%: train loss: 81.21456909179688
Epoch 40000 40.0%: train loss: 77.97526550292969
Epoch 50000 50.0%: train loss: 74.84622955322266
Epoch 60000 60.0%: train loss: 72.31035614013672
Epoch 70000 70.0%: train loss: 70.0422134399414
Epoch 80000 80.0%: train loss: 67.69535064697266
Epoch 90000 90.0%: train loss: 65.37538146972656
Epoch 100000 100.0%: train loss: 62.86763000488281


# Testing

In [10]:
# Sets the module in evulation mode.
ffnet_model.eval()

y_pred = ffnet_model(X_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

Test loss after Training 77.04769897460938


In [11]:
from random import randrange
num = randrange(len(y_test))
print(num, y_pred.squeeze()[num], y_test[num])

241 tensor(8.8664, grad_fn=<SelectBackward>) tensor(8.3212)
